# scrapy分布式部署

将代码放到远程主机是通过拷贝或者git的方式，考虑到多台远程主机的情况，这种方式就比较麻烦，那有没有好用的方法呢？这里其实可以通过scrapyd,

下面是这个[scrapyd的github地址：https://github.com/scrapy/scrapyd](https://github.com/scrapy/scrapyd)

安装scrapyd：`pip install scrapyd`

在另外一台ubuntu linux虚拟机中同样安装scrapy以及scrapyd等包，保证所要运行的爬虫需要的包都完成安装，这样就有了两台linux.

这里有个小问题需要注意: 默认scrapyd启动是通过scrapyd就可以直接启动，这里bind绑定的ip地址是127.0.0.1端口是：6800，为了其他虚拟机访问将ip地址设置为0.0.0.0

scrapyd的配置文件：`/usr/local/lib/python3.5/dist-packages/scrapyd/default_scrapyd.conf`

### 关于部署

如何通过scrapyd部署项目，这里官方文档提供一个地址：`https://github.com/scrapy/scrapyd-client`，即通过scrapyd-client进行操作

这里的scrapyd-client主要实现以下内容：
1. 把我们本地代码打包生成egg文件
2. 根据我们配置的url上传到远程服务器上

将我们本地的scrapy项目中scrapy.cfg配置文件进行配置：

```
[deploy]
url =  http://192.168.1.9:6800/addversion.json
project = zhihu_user
```

其实还可以设置用户名和密码，不过这里没什么必要，只设置了url
这里设置url一定要注意：`url = http://192.168.1.9:6800/addversion.json`, 最后的addversion.json不能少

我们在本地安装`pip install scrapy_client`,安装完成后执行：`scrapyd-deploy`, 看到status:200表示已经成功b

### 关于常用操作API

listprojects.json: 列出上传的项目列表
listversions.json: 列出有某个上传项目的版本
schedule.json: 远程任务的启动
listjobs.json: 列出所有的jobs任务
cancel.json: 取消所有运行的任务

我相信看了上面这几个方法你一定会觉得真不方便还需要输入那么长,
所以有人替你干了件好事把这些API进行的再次封装：`https://github.com/djm/python-scrapyd-api`

关于python-scrapyd-api

同时当启动完成后，就可以通过页面查看jobs

### 关于python-scrapyd-api

该模块可以让我们直接在python代码中进行上述那些api的操作

首先先安装该模块：`pip install python-scrapyd-api`

使用方法如下,这里只演示了简单的例子，其他方法其实使用很简单按照规则写就行：
```python
from scrapyd_api import ScrapydAPI


scrapyd = ScrapydAPI('http://192.168.1.9:6800')
res = scrapyd.list_projects()
res2 = scrapyd.list_jobs('zhihu_user')
print(res)
print(res2)
```

```
Cancel a scheduled job
scrapyd.cancel('project_name', '14a6599ef67111e38a0e080027880ca6')

Delete a project and all sibling versions
scrapyd.delete_project('project_name')

Delete a version of a project
scrapyd.delete_version('project_name', 'version_name')

Request status of a job
scrapyd.job_status('project_name', '14a6599ef67111e38a0e080027880ca6')

List all jobs registered
scrapyd.list_jobs('project_name')

List all projects registered
scrapyd.list_projects()

List all spiders available to a given project
scrapyd.list_spiders('project_name')

List all versions registered to a given project
scrapyd.list_versions('project_name')

Schedule a job to run with a specific spider
scrapyd.schedule('project_name', 'spider_name')

Schedule a job to run while passing override settings
settings = {'DOWNLOAD_DELAY': 2}

Schedule a job to run while passing extra attributes to spider initialisation
scrapyd.schedule('project_name', 'spider_name', extra_attribute='value')
```